## Train and deploy with private registry on Kubeflow from Notebooks

### In order to use a private registry in fairing append builder on the Jupyter notebook, two settings are required.
- Copy the .docker/config.json containing the private registry connection information to /home/jovyan/.docker/config.json on Jupyter-notebook container for docker-image-push by the append builder.
    ```
    # under docker client environ.
    $ docker login <private_registry>
    $ cat ~/.docker/config.json
      
    # in notebook console
    $ mkdir /home/jovyan/.docker
    $ vi /home/jovyan/.docker/config.json
    ```
-  Patch imagePullSecrets on the service account so that the deployer can pull the private registry image.
    ```
    # in current context(or in notebook console)
    $ kubectl create secret generic regcred --from-file=.dockerconfigjson=/home/${USER}/.docker/config.json \
    --type=kubernetes.io/dockerconfigjson -n anonymous  
    $ kubectl patch serviceaccount default -p '{"imagePullSecrets": [{"name": "regcred"}]}' -n anonymous
    ```

In [ ]:
import os
from kubeflow import fairing
import time

In [ ]:
DOCKER_REGISTRY = '<private_registry>'
fairing.config.set_builder('append',base_image='gcr.io/kubeflow-images-public/tensorflow-1.14.0-notebook-cpu:v0.7.0', registry=DOCKER_REGISTRY, push=True)
fairing.config.set_deployer('job')

In [ ]:
def train():
    print("hello world!")

In [ ]:
# Training on notebook 
train()

In [ ]:
# Training on current kubernetes context.
remote_train = fairing.config.fn(train)
remote_train()